# Importation of all libraries

In [1]:
# Import for libraries

from time import sleep # To prevent overwhelming the server between connections
import pandas as pd # For converting results to a dataframe and bar chart plots
import numpy as np
from selenium import webdriver # Because we get a dynamic website and we need to make some actions on it
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait



## To scrap this website, I decided to do it by type because they were no type categories on this page of the website. 
 
  So we need to run this for each category to grap each dataset

# Setup of my driver

In [7]:
## url_page is use to define the link for each type of property in the list_property

list_type = ['houses','condos','townhouses']

def url_page(self):
    
    url = f'https://www.zolo.ca/toronto-real-estate/{self}'  
    return url
driver = webdriver.Chrome('C:/Users/david/Desktop/Python/chromedriver.exe')
driver.get(url_page(list_type[1]))


C:\Users\david\AppData\Local\Temp/ipykernel_12836/1927175426.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/david/Desktop/Python/chromedriver.exe')


# Sign In Function
Without this function you will not be allow to see all the data on the website.


    

In [8]:
# click on signIn(The page need to be in a large format)

Sign = driver.find_element_by_xpath('//*[@id="drawer-menu"]/div/nav/div[3]/button[2]').click()  
    
# Type your email adress( wait 10s until this place is clickable)
    
email_address = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#email")))
email_address.clear()
email_address.send_keys('Your_email')
sleep(10)
# Click on Show me

Show_me = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#submitEmail"))).click()

# Enter acess code( *** for my account)

pin = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#pin")))

pin.clear()
pin.send_keys('access_code')# I need to run it when I am writing my code

sleep(10)
Sign_In = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > main > div > div > section > form > button"))).click()


C:\Users\david\AppData\Local\Temp/ipykernel_12836/26744304.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Sign = driver.find_element_by_xpath('//*[@id="drawer-menu"]/div/nav/div[3]/button[2]').click()


## Extraction of the data


In [9]:
# A create this fonction to transform this  patern of str in a dictionnary
# '$299\n  6684bed uwcbebath 46464sqft' --->{'price': '299', 'bed': '6684', 'bath': 'uwcbe', 'size': '46464'}

def parse(self):
    
    result = {}
    
    try:
        result['price'] = self.split('\n')[0][1:]
    except:
        result['price'] = 'Nan'
    try:
        result['bed'] = self.split('\n')[1].split('bed')[0].strip()
    except:
        result['bed'] = 'Nan'
    try:
        result['bath'] = self.split('\n')[1].split('bed')[1].split('bath')[0].strip()
    except:
        result['bath'] = 'Nan'
    try:
        result['size'] = self.split('\n')[1].split('bed')[1].split('bath')[1].split('sqft')[0].strip()
    except:
        result['size'] = 'Nan'
    return result

parse('$299\n  6684bed uwcbebath 46464sqft')

{'price': '299', 'bed': '6684', 'bath': 'uwcbe', 'size': '46464'}

In [10]:
# Number of pages. Every pages has 24 articles.

num_articles = driver.find_element_by_xpath('//*[@id="listing_container"]/section[1]/div/h3').text

nb_articles = int(num_articles.split(' ')[0])

nb_pages = nb_articles//24 +1  #each page avec 24 articles

nb_pages


C:\Users\david\AppData\Local\Temp/ipykernel_12836/3558092548.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  num_articles = driver.find_element_by_xpath('//*[@id="listing_container"]/section[1]/div/h3').text


74

In [11]:
List_card = []

num_articles = driver.find_element_by_xpath('//*[@id="listing_container"]/section[1]/div/h3').text

nb_articles = int(num_articles.split(' ')[0])

for i in range(nb_pages):
    #get the data
    
    sleep(10)
    Card_lists = driver.find_element_by_xpath('//*[@id="gallery"]/div').find_elements_by_tag_name('article')
    
    for Card_list in Card_lists:
        try:

            Card_values = Card_list.find_element_by_tag_name('ul').text

            Values = parse(Card_values)

            Card_street = Card_list.find_element_by_class_name('street').text

            Card_city = Card_list.find_element_by_class_name('city').text

            Card_province = Card_list.find_element_by_class_name('province').text

            Card_neighbourhood = Card_list.find_element_by_class_name('neighbourhood').text

            Cd = {
                'Province'      : Card_province,
                'City'          : Card_city,
                'Street'        : Card_street,
                'Neighbourhood' : Card_neighbourhood[1:],# remove the point           
                'Price'         : Values['price'],
                'Bed'           : Values['bed'],
                'Bath'          : Values['bath'],
                'Size'          : Values['size']    
            }



            List_card.append(Cd)

        except: 
            pass
    
    print(i)# to make sure that everything it alright
    sleep(7)
    try:
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#listing_container > section.supplementary-nav.xs-my6.xs-flex.xs-flex-column.xs-flex-align-center > nav.xs-hide.md-flex > a.button.button--mono.button--large.xs-flex.xs-flex-align-center"))).click()
    except:
        pass 

List_card


C:\Users\david\AppData\Local\Temp/ipykernel_12836/2529522321.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  num_articles = driver.find_element_by_xpath('//*[@id="listing_container"]/section[1]/div/h3').text
C:\Users\david\AppData\Local\Temp/ipykernel_12836/2529522321.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  Card_lists = driver.find_element_by_xpath('//*[@id="gallery"]/div').find_elements_by_tag_name('article')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73


[{'Province': 'ON',
  'City': 'Toronto',
  'Street': '216-1350 Kingston Road',
  'Neighbourhood': 'Birchcliffe-Cliffside',
  'Price': '1,149,000',
  'Bed': '2',
  'Bath': '2',
  'Size': '900-999'},
 {'Province': 'ON',
  'City': 'Toronto',
  'Street': '403-1486 Bathurst Street',
  'Neighbourhood': 'Humewood-Cedarvale',
  'Price': '629,990',
  'Bed': '1+1',
  'Bath': '1',
  'Size': '600-699'},
 {'Province': 'ON',
  'City': 'Toronto',
  'Street': '99 Woody Vine Way',
  'Neighbourhood': 'Bayview Village',
  'Price': '800,000',
  'Bed': '3+1',
  'Bath': '3',
  'Size': '1000-1199'},
 {'Province': 'ON',
  'City': 'Toronto',
  'Street': "PH12-118 Merchants' Wharf",
  'Neighbourhood': 'Waterfront Communities C8',
  'Price': '6,450,000',
  'Bed': '3+1',
  'Bath': '5',
  'Size': '4000-4249'},
 {'Province': 'ON',
  'City': 'Toronto',
  'Street': 'TH 101-70 Annie Craig Drive',
  'Neighbourhood': 'Mimico',
  'Price': '1,199,000',
  'Bed': '2+1',
  'Bath': '3',
  'Size': '1200-1399'},
 {'Province': '

# Create my Data frame

In [12]:
data = pd.DataFrame(List_card)

data

,Province,City,Street,Neighbourhood,Price,Bed,Bath,Size
0,ON,Toronto,216-1350 Kingston Road,Birchcliffe-Cliffside,"1,149,000",2,2,900-999
1,ON,Toronto,403-1486 Bathurst Street,Humewood-Cedarvale,"629,990",1+1,1,600-699
2,ON,Toronto,99 Woody Vine Way,Bayview Village,"800,000",3+1,3,1000-1199
3,ON,Toronto,PH12-118 Merchants' Wharf,Waterfront Communities C8,"6,450,000",3+1,5,4000-4249
4,ON,Toronto,TH 101-70 Annie Craig Drive,Mimico,"1,199,000",2+1,3,1200-1399
...,...,...,...,...,...,...,...,...
1751,ON,Toronto,918-251 Manitoba Street,Mimico,"700,000",1+1,1,500-599
1752,ON,Toronto,401-3 Marine Parade Drive,Mimico,"525,000",2+1,2,1000-1199
1753,ON,Toronto,UPH3-80 John Street,Waterfront Communities C1,"2,498,900",2+1,2,2250-2499
1754,ON,Toronto,TH2-12 Macpherson Avenue,Annex,"2,645,000",2,4,2500-2749


# Data Cleaning
    - Price as a float
    - Number of bed: 2+1 = 3
    - Number of rooms: bed + bath 

In [13]:
# Convert each column the best df

data = data.convert_dtypes()

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756 entries, 0 to 1755
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Province       1756 non-null   string
 1   City           1756 non-null   string
 2   Street         1756 non-null   string
 3   Neighbourhood  1756 non-null   string
 4   Price          1756 non-null   string
 5   Bed            1756 non-null   string
 6   Bath           1756 non-null   string
 7   Size           1756 non-null   string
dtypes: string(8)
memory usage: 109.9 KB


In [29]:
# Price as float

for i in range(len(data['Price'])):# remove the comma
    data['Price'][i] = ''.join(data['Price'][i].split(','))

data['Price'] = data['Price'].astype(np.float64)

data['Price']

AttributeError: 'numpy.float64' object has no attribute 'split'

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1756 entries, 0 to 1755
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Province       1756 non-null   string 
 1   City           1756 non-null   string 
 2   Street         1756 non-null   string 
 3   Neighbourhood  1756 non-null   string 
 4   Price          1756 non-null   float64
 5   Bed            1756 non-null   string 
 6   Bath           1756 non-null   string 
 7   Size           1756 non-null   string 
dtypes: float64(1), string(7)
memory usage: 109.9 KB


In [17]:
# list of the number of room in each article

room = []

for i in range(len(data)):
    
    try:
        a = int(data['Bed'][i].split('+')[0])
    except:
        a = 0
    try:
        b = int(data['Bed'][i].split('+')[1])
    except:
        b = 0
    try:
        c = int(data['Bath'][i])
    except:
        c = 0
        
    x = a+b+c
    
    room.append(x)

room 

[4,
 3,
 7,
 9,
 6,
 4,
 2,
 2,
 6,
 4,
 2,
 2,
 4,
 3,
 2,
 2,
 4,
 4,
 4,
 4,
 3,
 2,
 2,
 3,
 4,
 5,
 4,
 5,
 2,
 1,
 4,
 3,
 4,
 5,
 5,
 4,
 4,
 6,
 4,
 3,
 4,
 3,
 3,
 2,
 4,
 5,
 6,
 4,
 3,
 2,
 4,
 5,
 5,
 2,
 5,
 5,
 4,
 6,
 3,
 3,
 4,
 5,
 3,
 5,
 4,
 5,
 8,
 4,
 3,
 4,
 7,
 4,
 3,
 3,
 5,
 5,
 4,
 3,
 5,
 6,
 3,
 2,
 2,
 4,
 5,
 4,
 4,
 5,
 4,
 7,
 4,
 2,
 4,
 5,
 4,
 8,
 2,
 3,
 3,
 6,
 3,
 3,
 3,
 4,
 4,
 5,
 5,
 2,
 3,
 3,
 3,
 6,
 2,
 3,
 4,
 6,
 4,
 5,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 5,
 5,
 2,
 4,
 2,
 2,
 5,
 2,
 2,
 3,
 6,
 3,
 2,
 3,
 4,
 4,
 4,
 2,
 5,
 3,
 4,
 5,
 3,
 3,
 5,
 2,
 5,
 2,
 4,
 4,
 2,
 5,
 3,
 3,
 5,
 2,
 4,
 1,
 4,
 1,
 5,
 3,
 7,
 3,
 5,
 4,
 2,
 3,
 3,
 3,
 5,
 1,
 3,
 5,
 4,
 5,
 5,
 3,
 6,
 2,
 4,
 3,
 2,
 4,
 4,
 1,
 6,
 5,
 4,
 4,
 6,
 2,
 2,
 2,
 4,
 5,
 2,
 5,
 8,
 5,
 5,
 4,
 5,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 2,
 4,
 2,
 3,
 4,
 4,
 4,
 2,
 4,
 4,
 4,
 4,
 5,
 3,
 4,
 4,
 4,
 3,
 5,
 3,
 2,
 5,
 4,
 2,
 4,
 3,
 5,
 5,
 5,
 2,
 3,
 3,
 5,
 5,
 3,


In [18]:
# Importation in my data frame

data['room'] = room

data['room']

0       4
1       3
2       7
3       9
4       6
       ..
1751    3
1752    5
1753    5
1754    6
1755    7
Name: room, Length: 1756, dtype: int64

In [20]:
data

,Province,City,Street,Neighbourhood,Price,Bed,Bath,Size,room
0,ON,Toronto,216-1350 Kingston Road,Birchcliffe-Cliffside,1149000.0,2,2,900-999,4
1,ON,Toronto,403-1486 Bathurst Street,Humewood-Cedarvale,629990.0,1+1,1,600-699,3
2,ON,Toronto,99 Woody Vine Way,Bayview Village,800000.0,3+1,3,1000-1199,7
3,ON,Toronto,PH12-118 Merchants' Wharf,Waterfront Communities C8,6450000.0,3+1,5,4000-4249,9
4,ON,Toronto,TH 101-70 Annie Craig Drive,Mimico,1199000.0,2+1,3,1200-1399,6
...,...,...,...,...,...,...,...,...,...
1751,ON,Toronto,918-251 Manitoba Street,Mimico,700000.0,1+1,1,500-599,3
1752,ON,Toronto,401-3 Marine Parade Drive,Mimico,525000.0,2+1,2,1000-1199,5
1753,ON,Toronto,UPH3-80 John Street,Waterfront Communities C1,2498900.0,2+1,2,2250-2499,5
1754,ON,Toronto,TH2-12 Macpherson Avenue,Annex,2645000.0,2,4,2500-2749,6


In [19]:
data['Size'].value_counts()

600-699         285
700-799         208
1000-1199       197
500-599         185
800-899         172
1200-1399       165
900-999         143
1400-1599       116
0-499           101
1600-1799        53
1800-1999        34
2000-2249        26
2250-2499        18
2750-2999         9
2500-2749         9
3000-3249         7
5000+             6
3250-3499         5
4000-4249         3
4250-4499         3
4500-4749         2
501 to 1000       2
Nan               2
3750-3999         1
3500-3749         1
1001 to 1500      1
1500-2000         1
4750-4999         1
Name: Size, dtype: Int64

In [21]:
# cleaning Data['Size']

data['Size'].replace('0 to 500','0-500',inplace = True)
data['Size'].replace('501 to 1000','501-1000',inplace = True)
data['Size'].replace('1001 to 1500','1000-1500',inplace = True)
data['Size'].replace('1501 to 2000','1500-2000',inplace = True)
data['Size'].replace('16-30 Years Old','0-0',inplace = True)
data['Size'].replace('6-15 Years Old','0-0',inplace = True)
data['Size'].replace('100+ Years Old','0-0',inplace = True)
data['Size'].replace('0-5 Years Old','0-0',inplace = True)
data['Size'].replace('Nan','0-0',inplace = True)
data['Size'].replace('New','0-0',inplace = True)
data['Size'].replace('51-99 Years Old','0-0',inplace = True)
data['Size'].replace('1100-1500','1000-1500',inplace = True)
data['Size'].replace('1100-1500','1000-1500',inplace = True)
data['Size'].replace('31-50 Years Old','0-0',inplace = True)
data['Size'].replace('1800-1999','1500-2000',inplace = True)
data['Size'].replace('2001 to 3000','2500-3000',inplace = True)

data['Size'].value_counts()



600-699      285
700-799      208
1000-1199    197
500-599      185
800-899      172
1200-1399    165
900-999      143
1400-1599    116
0-499        101
1600-1799     53
1500-2000     35
2000-2249     26
2250-2499     18
2750-2999      9
2500-2749      9
3000-3249      7
5000+          6
3250-3499      5
4000-4249      3
4250-4499      3
4500-4749      2
501-1000       2
0-0            2
3750-3999      1
3500-3749      1
1000-1500      1
4750-4999      1
Name: Size, dtype: Int64

In [22]:
data.to_csv("condos_fdata.csv")
